###  Sparse GP Regression- FITC prior for sampling over f.


This notebook uses the FITC prior to sample over the latent function values. The integration over inducing 
outputs(u) can be done analytically, and we can directly sample from the multivariate Gaussian. Data to be used will be Snelson 1D dataset.

In [1]:
%matplotlib inline
import  matplotlib
import matplotlib.pyplot as plt
import pystan

In [2]:
import numpy as np

In [3]:
import scipy.io as sio

In [4]:
df = sio.loadmat('snelson1d.mat')

In [5]:
X = df['X']
Y = df['Y']
X_t = df['tX']
Y_t = df['tY']

In [6]:
M = 20

In [7]:
# X_u = kmm_init(X)
# M = len(X_u)

In [8]:
X_u_noisy = np.linspace(0,6, M)[:, None]
# X_u_noisy = X_u + 0.1*np.random.rand(*X_u.shape) 

### Code

In [9]:
stan_code_fitc_int ="""
data {
    int<lower=1> N;
    int<lower=1> M;
    real x[N];
    vector[N] y;
    real xu[M];

}
transformed data {
    real delta = 1e-6;
//     real rho = 0.9;
//     real alpha = 0.85;
//     real sigma = 0.12;
}

parameters {
    vector[N] f;
    
    real<lower=1e-6> sigma;
     real <lower=1e-6> alpha;
     real<lower=1e-6> rho;
    
    vector[M] eta2;
    
}

model {
    vector[N] f_mean;
    matrix[N, N] f_cov;
    matrix[N, N] S;
    matrix[N, N] A;
    matrix[N, N] A_inv;
    matrix [N,N] K_cov;
    matrix [N,N] Qff;
    matrix [N,N] L_fitc;
        
    {
        matrix[M, M] L_U;
        matrix[M, M] Kuu = cov_exp_quad(xu, alpha, rho);
        matrix[N, M] Kfu = cov_exp_quad(x, xu, alpha, rho);
        matrix[N, N] K = cov_exp_quad(x, alpha, rho);
        vector[M] Kuu_inv_u;
        matrix[M, N] v_pred;
        vector[M] u;
        
         for (m in 1:M)
             Kuu[m, m] = Kuu[m, m] + delta;
            
        L_U = cholesky_decompose(Kuu);
        u = L_U * eta2;
               
        Kuu_inv_u = mdivide_left_tri_low(L_U, eta2);
        f_mean = Kfu*Kuu_inv_u;
        

        // f_cov = K - Kfu*(Kuu_inv*Kfu');            
        v_pred = mdivide_left_tri_low(L_U, Kfu');
        
        Qff = v_pred'*v_pred;
        S = K - Qff;
        
        for (i in 1:N){
            for(j in 1:N){
                A[i,j] = 0.0;
                A_inv[i,j] = 0.0;
            }
            A[i,i] = S[i,i];
            A_inv[i,i] = 1./ S[i,i];
        }
        
        K_cov = A + Qff;
         
        L_fitc = cholesky_decompose(K_cov);
        
    }
  
#     rho ~ inv_gamma(5, 5);
#     alpha ~ normal(0, 1);

// strongly informative priors, but with these the covariance matrix will be more well behaved in my experience.
// The mean of the Gaussian priors is close to the optimised MAP value of the hyperparameters.

     rho ~ normal(0.90, 0.25);
     alpha ~ normal(0.85, 0.25);

    sigma ~ normal(0, 0.35);
    
    f ~ multi_normal_cholesky(f_mean, L_fitc);
    y ~ normal(f, sigma);
}

"""


In [10]:
stan_model_fitc = pystan.StanModel(model_code=stan_code_fitc_int)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_35c8a0f1801d4931198572edc53d1a9b NOW.


In [12]:
gp_full_data2 = {'N':200, 'x':X.flatten(), 'y':Y.flatten(), 'M':20, 'xu':X_u_noisy.flatten()}

In [13]:
fit_fitc = stan_model_fitc.sampling(data=gp_full_data2, iter=800, chains=2)

/u/26/dhakaa1/unix/anaconda2/envs/stanpy35/lib/python3.5/site-packages/pystan/misc.py:399: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif np.issubdtype(np.asarray(v).dtype, float):


In [ ]:
print(fit_fitc)

In [ ]:
samples = fit_fitc.extract(permuted=True)
f = samples['f']
f_mean = np.mean(f, axis=0)
sigma = samples['sigma']
l = samples['rho']
sigvar = samples['alpha']
sigma_mean = np.mean(sigma, axis=0)

In [ ]:
sort_idx = np.argsort(X.ravel())

plt.plot(X[sort_idx], Y[sort_idx], 'r.')
# plt.plot(X[sort_idx], f[-1, sort_idx], 'y.');
plt.plot(X[sort_idx], f[-100, sort_idx], 'g.');
# plt.plot(X[sort_idx], f_mean[sort_idx], 'b.');
# plt.plot(X[sort_idx], sigma[-1, sort_idx], 'b.');

In [ ]:
plt.plot(X,Y, 'ro')

In [ ]:
eta = samples['eta2']
print(eta.shape)
plt.plot(f[:, 0])
plt.plot(f[:, 1])
plt.plot(f[:, 2])


In [ ]:
plt.hist(sigma, range=[0, 0.8]);

In [ ]:
# plt.hist(l, range=[0, 2]);
plt.hist(sigvar, range=[0, 2]);

In [ ]:
sigma_mean = np.mean(samples_fitc['sigma'])
sigma_median = np.nanmedian(samples_fitc['sigma'])

l_mean = np.mean(samples_fitc['rho'])
l_median = np.nanmedian(samples_fitc['rho'])

sigvar_mean = np.mean(samples_fitc['alpha'])
sigvar_median = np.nanmedian(samples_fitc['alpha'])

print("mean of noise variance: %.2f" % sigma_mean)
print("median of noise variance: %.2f"% sigma_median)
print("mean of lengthscale: %.3f"% l_mean)
print("median of lengthscale: %.3f"% l_median)
print("mean of signal variance: %.3f"% sigvar_mean)
print("median of signal variance: %.3f"% sigvar_median)

In [ ]:
eta_fitc = samples_fitc['eta2']
plt.plot(eta_fitc[:, 0])
plt.plot(eta_fitc[:, 1])
plt.plot(eta_fitc[:, 2])
plt.plot(eta_fitc[:, 5])

NameError: name 'plt' is not defined